In [1]:
import os
import sys
import csv
import boto3
import botocore
from retrying import retry

# configuration
# s3_bucket = 'adomni-placeiq-sync/neon_query_temp'  # S3 Bucket name
s3_bucket = 'aws-athena-query-results-734644148268-us-east-1'
s3_ouput  = 's3://'+ s3_bucket   # S3 Bucket to store results
database  = 'default'  # The database to which the query belongs

In [2]:
# init clients
athena = boto3.client('athena')
s3 = boto3.resource('s3')

In [3]:
# @retry(stop_max_attempt_number = 100,
#     wait_exponential_multiplier = 300,
#     wait_exponential_max = 1 * 60 * 1000)
@retry(
    stop_max_delay = 1000 * 60 * 3,  # Try it until 3 min passes. 
    wait_fixed = 200 # Sleep 0.2 sec
      )
def poll_status(_id):
    result = athena.get_query_execution(QueryExecutionId=_id)
    state  = result['QueryExecution']['Status']['State']

    if state == 'SUCCEEDED':
        return result
    elif state == 'FAILED':
        return result
    else:
        raise Exception

In [4]:
def run_query(query, database, s3_output):
    response = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration = {
            'OutputLocation': s3_output,
            'EncryptionConfiguration': {
                'EncryptionOption': 'SSE_S3'
            }
        }
    )
    
    query_execution_id = response['QueryExecutionId']
    result = poll_status(query_execution_id)
#     print(result)
    
    
    if result['QueryExecution']['Status']['State'] == 'SUCCEEDED':
        print()
        print("Query SUCCEEDED: ")
        print(query_execution_id)
                
        s3_key = query_execution_id + '.csv'
        filename = query_execution_id + '.csv'
        
        # Download result file. 
        try:
            s3.Bucket(s3_bucket).download_file(s3_key, 'data/' + filename)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
        
        # Read file to array. 
        rows = []
        with open('data/' + filename) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                rows.append(row)

        # Delete result file.
#         if os.path.isfile(filename):
#             os.remove(filename)

        return rows

    if result['QueryExecution']['Status']['State'] == 'FAILED':
        print()
        print("Query FAILED: ")
        print(result['QueryExecution']['Status']['StateChangeReason'])


In [8]:
# SQL Query to execute

# Database: 
# location_data

# Tables: 
#
# hist_20190817_billboard_audiences: ['billboard_id', 'dma_name', 'audience', 'audience_index']
# 
# hist_20190817_billboard_devices: ['billboard_id', 'mobile_device_id']
# 
# hist_20190817_device_audiences: ['mobile_device_id', 'audience']
# 
# billboard_audiences_partitioned
# billboard_devices_partitioned
# adomni_audience_segment
# current_adomni_map


# query = ("""
#     SELECT * 
#     FROM location_data.hist_20190817_billboard_devices 
#     WHERE mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e';
# """)

# query = ("""
#      select * 
#      from location_data.billboard_audiences_partitioned
#      where dt = 20190817
#      limit 5;
# """)

# query = ("""
#      select * 
#      from location_data.hist_20190817_device_audiences
#      limit 100
# """)

# Q1. How many billboards was device '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e' seen in front of?  
# (Answer = 6) What is the query to get this value?
# 
# query = ("""
#       select count(distinct billboard_id)
#       from location_data.hist_20190817_billboard_devices
#       where mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e'
#  """)


# Q2. What audiences was the same mobile device seen in?
# 
# query = ("""
#       select distinct audience
#       from location_data.hist_20190817_device_audiences
#       where mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e'
#  """)


# Q3. What billboard has the highest number of mobile devices in the "Demographic->Income->200KPlus" segment?
# 
# query = ("""
# SELECT
#     billboard_id, 
#     count(a.mobile_device_id) mycount
# FROM location_data.hist_20190817_billboard_devices a
# INNER JOIN (
#     SELECT 
#         mobile_device_id
#     FROM 
#         location_data.hist_20190817_device_audiences
#     WHERE audience = 'Demographic->Income->200KPlus' 
# ) b ON a.mobile_device_id = b.mobile_device_id  
# GROUP BY 
#   billboard_id
# ORDER BY
#   mycount desc
# """)


# Q4. What are those device ids?  


# Q5. What other boards do those devices frequent?
#
# query = ("""
# select
#   distinct billboard_id
# from
#   location_data.hist_20190817_billboard_devices
# where
#   mobile_device_id in (
#     select
#       mobile_device_id
#     from
#       location_data.hist_20190817_billboard_devices
#     where
#       billboard_id = 'f2a8fea85d723bd01600c31a307d1e81'
#   )
# """)


# Part of our algorithm. 
# How many mobile devices for each audience segment in the given billboard?
# This query took a long time to get the result. 
#  
# query = ("""
# SELECT
#     a.billboard_id, 
#     b.audience, 
#     count(a.mobile_device_id) mycount
# FROM location_data.billboard_devices_partitioned a
# INNER JOIN location_data.device_audiences_partitioned b ON a.mobile_device_id = b.mobile_device_id  
# GROUP BY 
#     a.billboard_id, b.audience
# ORDER BY
#     mycount desc
#  """)


# How about this one using given audience segment id? And I use the result to get count for the given billboard_id. 
# The result will have count for each billboard_id. 
# 
# query = ("""
# SELECT
#     a.billboard_id, 
#     count(a.mobile_device_id) mycount
# FROM location_data.billboard_devices_partitioned a
# INNER JOIN (
#     SELECT 
#         mobile_device_id
#     FROM 
#         location_data.device_audiences_partitioned
#     WHERE audience = 'Demographic->Income->200KPlus' 
# ) b on a.mobile_device_id = b.mobile_device_id  
# GROUP BY 
#     billboard_id
# ORDER BY
#     mycount desc
# """)


# audience = 'Dining->Coffee->Starbucks'
# billboard_id = 'dbb561c792f78028f262e88ce95f857c'

# query = """
# SELECT
#     a.billboard_id, 
#     count(a.mobile_device_id) my_count
# FROM 
#     location_data.billboard_devices_partitioned a
# INNER JOIN (
#     SELECT 
#         mobile_device_id
#     FROM 
#         location_data.device_audiences_partitioned
#     WHERE 
#         audience = '%s' 
# ) b 
# ON 
#     a.mobile_device_id = b.mobile_device_id  
# WHERE
#     a.billboard_id = '%s'
# GROUP BY 
#     a.billboard_id;
# """ % (audience, billboard_id)


# adomni_audience_segment table 
# query = ("""
# SELECT
#     *
# FROM 
#     location_data.adomni_audience_segment
# """)


audience_id = '257'
billboard_id = 'dbb561c792f78028f262e88ce95f857c'

query = """
SELECT
    a.billboard_id, 
    count(a.mobile_device_id) my_count
FROM 
    location_data.billboard_devices_partitioned a
INNER JOIN
    location_data.device_audiences_partitioned b
ON 
    a.mobile_device_id = b.mobile_device_id
INNER JOIN
    location_data.adomni_audience_segment c
ON
    b.audience = c.placeiqid
WHERE
    a.billboard_id = '%s'
    AND
    c.id = '%s'
GROUP BY 
    a.billboard_id
""" % (billboard_id, audience_id)






print("Executing query: {}".format(query))
result = run_query(query, database, s3_ouput)

print("Results:")
print(result)



for key, value in result[0].items(): 
    print(key, value) 
    
    
    
count = result[0]['my_count']

print(count)

Executing query: 
SELECT
    a.billboard_id, 
    count(a.mobile_device_id) my_count
FROM 
    location_data.billboard_devices_partitioned a
INNER JOIN
    location_data.device_audiences_partitioned b
ON 
    a.mobile_device_id = b.mobile_device_id
INNER JOIN
    location_data.adomni_audience_segment c
ON
    b.audience = c.placeiqid
WHERE
    a.billboard_id = 'dbb561c792f78028f262e88ce95f857c'
    AND
    c.id = '257'
GROUP BY 
    a.billboard_id


Query SUCCEEDED: 
95dcacc7-1ed0-4c09-b4ef-25f2daebde1f
Results:
[OrderedDict([('billboard_id', 'dbb561c792f78028f262e88ce95f857c'), ('my_count', '1012')])]
billboard_id dbb561c792f78028f262e88ce95f857c
my_count 1012


In [10]:
import pandas as pd
from pandas import Series, DataFrame


# billboard_audience_count = pd.read_csv('https://aws-athena-query-results-734644148268-us-east-1.s3.amazonaws.com/f6872496-717c-479e-a4cb-e27a3e5cfdbe.csv')
# billboard_audience_count.head()

In [11]:
# audience_data = pd.read_csv('https://aws-athena-query-results-734644148268-us-east-1.s3.amazonaws.com/f6872496-717c-479e-a4cb-e27a3e5cfdbe.csv')
# billboard_audience_count.head()

In [41]:
def get_max_count(audience_id):
    # Take as input the result_max.csv here.
    max_counts = pd.read_csv('data/result_max.csv')
    # max_counts.head()
    # len(max_counts)
    max_count = max_counts[max_counts['id'] == audience_id]['max']

    return max_count.values[0]

m = get_max_count(257)
m

52707.0

In [35]:
max_counts = pd.read_csv('data/result_max.csv')
max_counts.head()

,id,max
0,1.0,2211.0
1,2.0,2226.0
2,3.0,3145.0
3,4.0,2553.0
4,5.0,2567.0
